In [1]:
import awkward as ak
from pyutils.pyprocess import Processor
from pyutils.pyprint import Print 
from pyutils.pyselect import Select

In [2]:
# # help(Processor)
# # nts.mu2e.CeEndpointOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.root
# import warnings
# warnings.filterwarnings("ignore")
# !getToken > /dev/null

In [10]:
file_name = "nts.mu2e.CeEndpointOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.001210_00000580.root"

branches = { 
    "evt" : ["run", "subrun", "event"],
    "crv" : ["crvcoincs.time", "crvcoincs.PEs", "crvcoincs.nHits", "crvcoincs.sectorType",
                "crvcoincs.angle", "crvcoincs.nLayers",  "crvcoincs.timeStart", "crvcoincs.timeEnd",
                "crvcoincs.pos.fCoordinates.fX", "crvcoincs.pos.fCoordinates.fY", "crvcoincs.pos.fCoordinates.fZ"],
    "trk" : ["trk.nactive", "trk.pdg", "trkqual.valid", "trkqual.result"],
    "trkfit" : ["trksegs", "trksegpars_lh"],
    "trkmc" : ["trkmcsim"]
} 

data = Processor(use_remote=True, location="disk").process_data(file_name=file_name, branches=branches)

[pyprocess] ⭐️ Initialised Processor:
	path = 'EventNtuple/ntuple'
	use_remote = True
	location = disk
	schema = root
	verbosity=1
[pyprocess] ✅ Completed process on nts.mu2e.CeEndpointOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.001210_00000580.root


In [11]:
def _append_array(data, arr, name):
    """Helper to append arrays/masks to dev field 
    This is useful for debugging and development
    Must be trk or trkfit level
    """
    try:
        if "dev" not in ak.fields(data):
            # Initialise dev field - structure will be determined by first array added
            data = ak.with_field(data, ak.zip({name: arr}, depth_limit=1), "dev")
        else: 
            # Add new field to existing 'dev' record
            new_dev = ak.with_field(data.dev, arr, name)
            data = ak.with_field(data, new_dev, "dev")
        return data
    except Exception as e:
        print("Error appending '{name}' to data array: {e}")
        raise e

In [12]:
def all_downstream(data):
    n = len(data)
    print(f"{n} events before cuts")
    
    # Sign
    q = -1 * data["trk"]["trk.pdg"] / abs(data["trk"]["trk.pdg"])
    data = _append_array(data, q, "q")

    # Tracker segments
    selector = Select()
    at_trk_front  = selector.select_surface(data["trkfit"], "TT_Front")
    at_trk_mid = selector.select_surface(data["trkfit"], "TT_Mid")
    at_trk_back  = selector.select_surface(data["trkfit"], "TT_Back")
    in_trk = (at_trk_front | at_trk_mid | at_trk_back)
    data = _append_array(data, in_trk, "in_trk")

    # Trajectory 
    downstream_seg = (-q * data["trkfit"]["trksegs"]["mom"]["fCoordinates"]["fZ"]) > 0
    downstream_trk = ak.all(~in_trk | downstream_seg, axis=-1)
    downstream_evt = ak.all(downstream_trk, axis=-1)
    data = _append_array(data, downstream_trk, "downstream_trk")
    data = _append_array(data, downstream_evt, "downstream_evt")

    data = _append_array(data, data["trkfit"]["trksegs"]["mom"]["fCoordinates"]["fZ"][in_trk], "p_z_trk")
    data = _append_array(data, -q*data["trkfit"]["trksegs"]["mom"]["fCoordinates"]["fZ"][in_trk], "q_x_p_z_trk")
    
    # Events with a downstream electon
    is_electron = data["trk"]["trk.pdg"] == 11
    downstream_electron_trk = is_electron & downstream_trk
    downstream_electron_evt = ak.any(downstream_electron_trk, axis=-1)
    data = _append_array(data, downstream_electron_trk, "downstream_electron_trk")
    data = _append_array(data, downstream_electron_evt, "downstream_electron_evt")
    
    # Return 
    Print(verbose=True).print_n_events(data["trk"], 1)
    Print(verbose=True).print_n_events(data["dev"], 1)

    return data

data = all_downstream(data)

1482 events before cuts
[pyselect] ✅ Returning mask for trksegs with sid = 0
[pyselect] ✅ Returning mask for trksegs with sid = 1
[pyselect] ✅ Returning mask for trksegs with sid = 2
[pyprint] ⭐️ Initialised Print with verbose = True and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
trk.nactive: [26, 25, 26, 20]
trk.pdg: [11, -11, 13, -13]
trkqual.valid: [True, True, True, True]
trkqual.result: [0.9, 0.1, 0.1, 0.0]
-------------------------------------------------------------------------------------

[pyprint] ⭐️ Initialised Print with verbose = True and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
q: [-1.0, 1.0, -1.0, 1.0]
in_trk: [[False, False, False, False, False, True, True, True, False, False, False], [False, False, False, False, False, False, True, True, True, False, False], [False, False, False, Fa

In [17]:
# upstream_electron_evts = data[(data["dev"]["downstream_trk"]) & (data["trk"]["trk.pdg"] == 11)]

downstream_electron_evts = data[data["dev"]["downstream_electron_evt"]]
downstream_evts = data[data["dev"]["downstream_evt"]]
exclusive = data[data["dev"]["downstream_electron_evt"] & ~data["dev"]["downstream_evt"]]


In [18]:
print(f"{len(downstream_electron_evts)} ({100*len(downstream_electron_evts)/len(data):.2f}%) downstream events")
print(f"{len(downstream_evts)} ({100*len(downstream_evts)/len(data):.2f}%) downstream events")
print(f"{len(exclusive)} ({100*len(exclusive)/len(data):.2f}%) exclusive events")

1479 (99.80%) downstream events
254 (17.14%) downstream events
1226 (82.73%) exclusive events


You can't be cutting 80% of events from the CE dataset, that's insane

In [22]:
# what = data[data["dev"]["downstream_evt"]]
# what = data[data["trk"]["downstream_electron_evt"]]

In [23]:
Print(True).print_n_events(downstream_electron_evts, 3)

[pyprint] ⭐️ Initialised Print with verbose = True and precision = 1
[pyprint] ⭐️ Printing 3 event(s)...

-------------------------------------------------------------------------------------
evt.run: 1210
evt.subrun: 580
evt.event: 7
crv.crvcoincs.time: []
crv.crvcoincs.PEs: []
crv.crvcoincs.nHits: []
crv.crvcoincs.sectorType: []
crv.crvcoincs.angle: []
crv.crvcoincs.nLayers: []
crv.crvcoincs.timeStart: []
crv.crvcoincs.timeEnd: []
crv.crvcoincs.pos.fCoordinates.fX: []
crv.crvcoincs.pos.fCoordinates.fY: []
crv.crvcoincs.pos.fCoordinates.fZ: []
trk.trk.nactive: [26, 25, 26, 20]
trk.trk.pdg: [11, -11, 13, -13]
trk.trkqual.valid: [True, True, True, True]
trk.trkqual.result: [0.9, 0.1, 0.1, 0.0]
trkfit.trksegs.mom.fCoordinates.fX: [[-27.0, 48.4, 71.7, 66.0, 58.8, 45.0, 37.0, 57.4, 82.6, 50.6, 74.0], [26.1, -47.7, -71.9, -66.6, -59.8, -51.6, -45.1, -37.0, -58.4, -72.1, -51.6], [-27.0, 48.9, 74.5, 69.5, 63.0, 55.0, 45.0, 37.0, 56.9, 76.9, 51.0, 74.7], [26.7, -47.3, -69.3, -63.3, -55.8, -44.

In [43]:
def all_downstream_with_trkqual(data):
    n = len(data)
    print(f"{n} events before cuts")
    good_trkqual = data["trk"]["trkqual.result"] > 0.2
    # Segments
    is_downstream_seg = data["trkfit"]["trksegs"]["mom"]["fCoordinates"]["fZ"] > 0
    # Tracks 
    all_downstream_segs = ak.all(is_downstream_seg, axis=-1)
    # Good tracks only
    all_good_downstream_segs = all_downstream_segs & good_trkqual
    # Append
    data = _append_array(data, is_downstream_seg, "is_downstream_seg")
    data = _append_array(data, all_downstream_segs, "all_downstream_segs")
    data = _append_array(data, good_trkqual, "good_trkqual")
    data = _append_array(data, all_good_downstream_segs, "all_good_downstream_segs")
    # Apply inverse cut
    data_cut = ak.copy(data)
    data_cut["trk"] = data["trk"][all_good_downstream_segs]
    has_trk = ak.num(data_cut["trk"]["trk.nactive"]) > 0
    # 
    data_cut = data_cut[has_trk]
    #########
    print(f"{len(data_cut)} events passing all downstream tracks cut")
    # Return 
    Print(verbose=True).print_n_events(data_cut["trk"], 1)
    Print(verbose=True).print_n_events(data_cut["evt"], 1)

all_downstream_with_trkqual(data)


1482 events before cuts
258 events passing all downstream tracks cut
[pyprint] ⭐️ Initialised Print with verbose = True and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
trk.nactive: [26]
trk.pdg: [11]
trkqual.valid: [True]
trkqual.result: [0.9]
-------------------------------------------------------------------------------------

[pyprint] ⭐️ Initialised Print with verbose = True and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
run: 1210
subrun: 580
event: 7
-------------------------------------------------------------------------------------

